In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline

In [ ]:
## load data
train = pd.read_csv('products_transactional.csv')

In [ ]:
## fill missing values
#products['customerID'].fillna('BBID_0000', inplace=True)
train['promotion_description'].fillna('no_promo', inplace=True)
train['Gender'].fillna('no_gender', inplace=True)
train['State'].fillna('no_state', inplace=True)
train['PinCode'].fillna(-1, inplace=True)
train['DOB'].fillna("1", inplace=True)

In [ ]:
#handling missing data for product_code
train = train[np.isfinite(train['product_code'])]

In [ ]:
#Correcting bad states
state_dict = {'MADHY PRADESH':'MADHYA PRADESH', 'TAMILNADU':'TAMIL NADU', 'MADHYA  PRADESH':'MADHYA PRADESH', 'HARAYANA':'HARYANA',
             'Jharkhand':'JHARKHAND','Tamilnadu':'TAMIL NADU','Tamil Nadu':'TAMIL NADU','Madhya Pradesh':'MADHYA PRADESH',
             'REST OF WEST BENGAL':'WEST BENGAL', 'west bengal':'WEST BENGAL','Uttar Pradesh':'UTTAR PRADESH', 'Delhi':'DELHI',
             'Bhopal':'BHOPAL','CHHATISGARH':'CHHATTISGARH','CHATTISGARH':'CHHATTISGARH', 'jharkhand':'JHARKHAND','Chandigarh':'CHANDIGARH',
             'UTTAR PRADESH WEST': 'UTTAR PRADESH','ODISHA':'ORISSA','MAHARASTRA':'MAHARASHTRA','madhya pradesh':'MADHYA PRADESH',
             'KARNATAK':'KARNATAKA','JAMMU and KASHMIR':'JAMMU AND KASHMIR','JAMMU KASHMIR':'JAMMU AND KASHMIR','Rajasthan':'RAJASTHAN',
             'east singhbhum':'JHARKHAND', 'ORRISA':'ORISSA','Andhra Pradesh':'ANDHRA PRADESH', 'UTTARANCHAL':'UTTARAKHAND',
             'Uttar pradesh':'UTTAR PRADESH','Maharashtra':'MAHARASHTRA','MP':'MADHYA PRADESH', 'UTTAR PRADESH EAST':'UTTAR PRADESH',
             'Punjab':'PUNJAB','maharashtra':'MAHARASHTRA','Karnataka':'KARNATAKA','M.P.':'MADHYA PRADESH','DAMAN':'DAMAN AND DIU',
             'HUBLI':'KARNATAKA','Tamil nadu':'TAMIL NADU','GUJRAT':'GUJARAT', 'Mp':'MADHYA PRADESH','Madhya pradesh':'MADHYA PRADESH',
             'West Bengal':'WEST BENGAL','Gujarat':'GUJARAT','UP':'UTTAR PRADESH','Chennai':'CHENNAI', 'm.p.':'MADHYA PRADESH',
             'kerala':'KERALA'}

train.replace({"State": state_dict}, inplace=True)

In [ ]:
train["transactionDate"] = pd.to_datetime(train["transactionDate"],format="%Y-%m-%d")

In [ ]:
train["month"] = pd.DatetimeIndex(train["transactionDate"]).month
train["day"] = pd.DatetimeIndex(train["transactionDate"]).day
train["dayofweek"] = pd.DatetimeIndex(train["transactionDate"]).dayofweek
train["year"] = pd.DatetimeIndex(train["transactionDate"]).year

In [ ]:
train.head()

## Frequency of order by week day

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x="dayofweek", data=train, color=color[0])
plt.ylabel('Count', fontsize=12)
plt.xlabel('Day of week', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Frequency of order by week day", fontsize=15)
plt.show()

So we see that the transactions are highest on Sunday, Wednesday and Saturday


## Frequency of order by day in a month

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x="day", data=train, color=color[0])
plt.ylabel('Count', fontsize=12)
plt.xlabel('Day of Month', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Frequency of order by days in a month", fontsize=15)
plt.show()

## Frequency of order by month

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x="month", data=train, color=color[0])
plt.ylabel('Count', fontsize=12)
plt.xlabel('Month', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Frequency of order by month", fontsize=15)
plt.show()

So we see the sales are highest around May-June


## Frequency of Orde by year

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x="year", data=train, color=color[0])
plt.ylabel('Count', fontsize=12)
plt.xlabel('Year', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Frequency of order by Year", fontsize=15)
plt.show()

2016 had the highest sales although for 2017, we only have data till July.


## Heatmap of Month Vs Day

In [ ]:
grouped_df = train.groupby(["month", "day"])["product_code"].aggregate("count").reset_index()
grouped_df = grouped_df.pivot('month', 'day', 'product_code')

plt.figure(figsize=(12,6))
sns.heatmap(grouped_df)
plt.title("HeatMap of amount of sales on Month vs Day")
plt.show()

## Top 20 most popular products

In [ ]:
cnt_srs = train['product_description'].value_counts().reset_index().head(20)
cnt_srs.columns = ['product_name', 'frequency_count']
cnt_srs

In [ ]:
cnt_srs = train['product_description'].value_counts().head(20)
plt.figure(figsize=(12,8))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8, color=color[5])
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Product', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()

## Transactions by State(Top 10)

In [ ]:
plt.figure(figsize=(10,10))
temp_series = train['State'].value_counts().head(10)
labels = (np.array(temp_series.index))
sizes = (np.array((temp_series / temp_series.sum())*100))
plt.pie(sizes, labels=labels, 
        autopct='%1.1f%%', startangle=200)
plt.title("State distribution", fontsize=15)
plt.show()

## Product analysis by TFIDF

Now we will see if the model will learn any useful information about the products from the order history of all users, maybe in the future this can be used as input to a classifier that recommends products.


In [ ]:
train["product_description"] = train["product_description"].astype(str)
train_tfidf = train.groupby("customerID").apply(lambda order: order['product_description'].tolist())
train_tfidf = train_tfidf.reset_index()
train_tfidf.columns = ['customerID','product_set']
train_tfidf.product_set = train_tfidf.product_set.astype(str)
train_tfidf.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(min_df=5, max_features=1000
                        , strip_accents='unicode',lowercase =True,
analyzer='word', token_pattern=r'\w+', use_idf=True, 
smooth_idf=True, sublinear_tf=True, stop_words = 'english')
tfidf.fit(train_tfidf['product_set'])

In [ ]:
from sklearn.decomposition import TruncatedSVD
text = tfidf.transform(train_tfidf['product_set'])
svd = TruncatedSVD(n_components=2)
text = svd.fit_transform(text)
text = pd.DataFrame(text)
text.columns = ['pf_0','pf_1']
text['customerID'] = train_tfidf.customerID
text.head()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(13,13))
plt.plot(text['pf_0'].head(50),text['pf_1'].head(50),'r*',label=text['customerID'].head(50))
for row in text.head(50).itertuples():
    plt.annotate('user_'+str(row.customerID), xy=(row.pf_0,row.pf_1), 
            xytext=(row.pf_0+0.01,row.pf_1+0.01)
            
                )

## Clustering

In [ ]:
cnt_srs = train['product_description'].value_counts().reset_index().head(500)
cnt_srs.columns = ['product_name', 'frequency_count']
cnt_srs

So we see that the top 5 products are: BB-CB-27X30X208SWG NEW, BB-CB-20X25X208SWG NEW, SUGAR MEDIUM LOOSE, BB-CB-20X25X168SWG-Suitable for ROI New and TOMATO LOOSE. We will confirm this hypothesis later with clustering

In [ ]:
train_top_500 = train[train['product_description'].isin(cnt_srs.product_name)].reset_index(drop=True)

In [ ]:
train_top_500.head()

In [ ]:
#crosstabbing cust_id and top 20 prods
cust_prod = pd.crosstab(train_top_500['customerID'], train_top_500['product_description'])
cust_prod.head(10)

In [ ]:
cust_prod.shape

In [ ]:
#I have selected 6 dimensions for the PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=6)
pca.fit(cust_prod)
pca_samples = pca.transform(cust_prod)

In [ ]:
ps = pd.DataFrame(pca_samples)
ps.head()

In [ ]:
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
tocluster = pd.DataFrame(ps[[4,1]])
print (tocluster.shape)
print (tocluster.head())

fig = plt.figure(figsize=(8,8))
plt.plot(tocluster[4], tocluster[1], 'o', markersize=2, color='blue', alpha=0.5, label='class1')

plt.xlabel('x_values')
plt.ylabel('y_values')
plt.legend()
plt.show()

## K-means

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

clusterer = KMeans(n_clusters=4,random_state=42).fit(tocluster)
centers = clusterer.cluster_centers_
c_preds = clusterer.predict(tocluster)
print(centers)

In [ ]:
#Print first 200 predictions
print (c_preds[0:200])

In [ ]:
import matplotlib
fig = plt.figure(figsize=(8,8))
colors = ['orange','blue','purple','green']
colored = [colors[k] for k in c_preds]
print (colored[0:10])
plt.scatter(tocluster[4],tocluster[1],  color = colored)
for ci,c in enumerate(centers):
    plt.plot(c[0], c[1], 'o', markersize=8, color='red', alpha=0.9, label=''+str(ci))

plt.xlabel('x_values')
plt.ylabel('y_values')
plt.legend()
plt.show()

## So now that we have a possible clustering for our customers, let's see if there are any more interesting patterns beneath.

In [ ]:
#Appending the cluster prediction column to the dataframe
clust_prod = cust_prod.copy()
clust_prod['cluster'] = c_preds

clust_prod.head(10)

In [ ]:
print (clust_prod.shape)
f,arr = plt.subplots(2,2,sharex=True,figsize=(15,15))

c1_count = len(clust_prod[clust_prod['cluster']==0])

#Find the mean for each cluster and plot it against the top 500 products
c0 = clust_prod[clust_prod['cluster']==0].drop('cluster',axis=1).mean()
arr[0,0].bar(range(len(clust_prod.drop('cluster',axis=1).columns)),c0)
c1 = clust_prod[clust_prod['cluster']==1].drop('cluster',axis=1).mean()
arr[0,1].bar(range(len(clust_prod.drop('cluster',axis=1).columns)),c1)
c2 = clust_prod[clust_prod['cluster']==2].drop('cluster',axis=1).mean()
arr[1,0].bar(range(len(clust_prod.drop('cluster',axis=1).columns)),c2)
c3 = clust_prod[clust_prod['cluster']==3].drop('cluster',axis=1).mean()
arr[1,1].bar(range(len(clust_prod.drop('cluster',axis=1).columns)),c3)
plt.show()

Let's check out what are the top 10 goods bought by people of each cluster.

In [ ]:
#Cluster c0
c0.sort_values(ascending=False)[0:10]

In [ ]:
#Cluster c1
c1.sort_values(ascending=False)[0:10

In [ ]:
#Cluster c2
c2.sort_values(ascending=False)[0:10]

In [ ]:
#Cluster c3
c3.sort_values(ascending=False)[0:10]

The interesting thing here is even though the 4 products: BB-CB-27X30X208SWG NEW, BB-CB-20X25X208SWG NEW, SUGAR MEDIUM LOOSE, BB-CB-20X25X168SWG-Suitable for ROI New confirm our hypothesis which we made previously about the top 5 products, TOMATO LOOSE does not fall under this category and is only present in one of the clusters.

What we can inspect here is if clusters differ in quantities and proportions, with respect of these goods, or if a cluster is characterized by some goods not included in this list.

In [ ]:
from IPython.display import display, HTML
cluster_means = [[c0['BB-CB-27X30X208SWG NEW'],c0['BB-CB-20X25X208SWG NEW'],c0['SUGAR MEDIUM LOOSE'], c0['BB-CB-20X25X168SWG-Suitable for ROI New'], c0['TOMATO LOOSE']],
                 [c1['BB-CB-27X30X208SWG NEW'],c1['BB-CB-20X25X208SWG NEW'],c1['SUGAR MEDIUM LOOSE'], c1['BB-CB-20X25X168SWG-Suitable for ROI New'], c1['TOMATO LOOSE']],
                 [c2['BB-CB-27X30X208SWG NEW'],c2['BB-CB-20X25X208SWG NEW'],c2['SUGAR MEDIUM LOOSE'], c2['BB-CB-20X25X168SWG-Suitable for ROI New'], c2['TOMATO LOOSE']],
                 [c3['BB-CB-27X30X208SWG NEW'],c3['BB-CB-20X25X208SWG NEW'],c3['SUGAR MEDIUM LOOSE'], c3['BB-CB-20X25X168SWG-Suitable for ROI New'], c3['TOMATO LOOSE']]]
cluster_means = pd.DataFrame(cluster_means, columns = ['BB-CB-27X30X208SWG NEW','BB-CB-20X25X208SWG NEW','SUGAR MEDIUM LOOSE','BB-CB-20X25X168SWG-Suitable for ROI New','TOMATO LOOSE'])
HTML(cluster_means.to_html())

The following table depicts the percentage these goods with respect to the other top 5 in each cluster.

It seems people of cluster 3 buy much more sugar than people from other clusters.

Another interesting observation is that people in cluster 2 and 3 buy a huge lot of TUR DAL even though it is not even there in the top 10 products. So this is where the unique properties of the clusters really shows up and this can be really used to a great use in later product recommendations.